In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import os
from VAERSFileDownloader import updateVAERSFiles
from datetime import datetime
from DateProvider import DateProvider
from InternationalVaersCovid19Provider import getInternationalVaersCovid19,get_international_VAERSVAX_VAERSSYMPTOMS_Covid19
from BatchCodeTableHtmlUpdater import updateBatchCodeTableHtmlFile
from BatchCodeTablePersister import createAndSaveBatchCodeTables

In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
years_from_2020_to_present = list(range(2020, datetime.now().year + 1))

In [ ]:
updateVAERSFiles(
        years = years_from_2020_to_present,
        workingDirectory = os.getcwd())

In [ ]:
international_VAERSVAX_Covid19, international_VAERSSYMPTOMS = get_international_VAERSVAX_VAERSSYMPTOMS_Covid19(years = years_from_2020_to_present)

In [ ]:
international_VAERSVAX_Covid19

In [ ]:
batchcodes = international_VAERSVAX_Covid19['VAX_LOT'].dropna().drop_duplicates().to_list()
batchcodes

In [ ]:
international_VAERSSYMPTOMS

In [ ]:
from SymptomByBatchcodeTableFactory import SymptomByBatchcodeTableFactory
symptomByBatchcodeTable = SymptomByBatchcodeTableFactory.createSymptomByBatchcodeTable(international_VAERSVAX_Covid19, international_VAERSSYMPTOMS)
symptomByBatchcodeTable

In [ ]:
symptomByBatchcodeTable.to_pickle('tmp/symptomByBatchcodeTable.pkl')

In [ ]:
symptomByBatchcodeTable = pd.read_pickle('tmp/symptomByBatchcodeTable.pkl')
symptomByBatchcodeTable

In [ ]:
# import cProfile
# from SymptomByBatchcodeTableFactory import SymptomByBatchcodeTableFactory
# cProfile.run('SymptomByBatchcodeTableFactory.createSymptomByBatchcodeTable(international_VAERSVAX_Covid19, international_VAERSSYMPTOMS)')
# # symptomByBatchcodeTable

In [ ]:
from SymptomHistogramByBatchcodeTableFactory import SymptomHistogramByBatchcodeTableFactory

symptomHistogramByBatchcodeTable = SymptomHistogramByBatchcodeTableFactory.createSymptomHistogramByBatchcodeTable(symptomByBatchcodeTable)
symptomHistogramByBatchcodeTable

In [ ]:
from HistogramTable2DictTableConverter import HistogramTable2DictTableConverter
dictByBatchcodeTable = HistogramTable2DictTableConverter.convertHistogramTable2DictTable(symptomHistogramByBatchcodeTable)
dictByBatchcodeTable

In [ ]:
from HistogramDescriptionPersister import HistogramDescriptionPersister

histogramDescriptionPersister = HistogramDescriptionPersister('../docs/data/histograms')
histogramDescriptionPersister.saveHistogramDescriptionsForBatchcodes(batchcodes[:100], dictByBatchcodeTable)

In [ ]:
print(json.dumps(json.loads(jsonActual), indent=2))

In [ ]:
jsonTable.to_excel('tmp/jsonTable.xlsx')

In [ ]:
filteredTable.to_excel('tmp/filteredTable.xlsx')

In [ ]:
from TestHelper import TestHelper
batchcode = '1808982'
symptomHistogramByBatchcodeTable = TestHelper.createDataFrame(
        columns = ['SYMPTOM_COUNT_BY_VAX_LOT'],
        data = [  ['{"Blood pressure orthostatic abnormal":5,"Chest discomfort":1}'],
                    ['{"Chest discomfort":2}']],
        index = pd.MultiIndex.from_tuples(
            names =   ['VAX_LOT1', 'VAX_LOT2'],
            tuples = [[batchcode,  'EW0175'],
                        ['015M20A',  batchcode]]))
symptomHistogramByBatchcodeTable

In [ ]:
import json
result= symptomHistogramByBatchcodeTable.to_json()
parsed = json.loads(result)
parsed
#print(json.dumps(parsed, indent=4))

In [ ]:
res = jsonTable.loc[('!D0181', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan')]['SYMPTOM_COUNT_BY_VAX_LOT']
res

In [ ]:
res.to_json()

In [ ]:
symptomHistogramByBatchcodeTable.loc[('!D0181', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan')].plot(kind='bar')

In [ ]:
symptomHistogramByBatchcodeTable.sort_values(by='SYMPTOM_COUNT_BY_VAX_LOT', ascending=False)

In [ ]:
table.loc[('!D0181', 'nan')]

In [ ]:
table.loc[('!D0181', 'nan')][:30].plot(kind='bar')

In [ ]:
df.groupby('VAX_LOT1')['SYMPTOMS']

In [ ]:
df[(df['VAX_LOT1'] != 'nan') & (df['VAX_LOT2'] != 'nan') & (df['VAX_LOT3'] != 'nan') & (df['VAX_LOT4'] != 'nan')& (df['VAX_LOT5'] != 'nan') & (df['VAX_LOT6'] != 'nan') & (df['VAX_LOT7'] != 'nan') & (df['VAX_LOT8'] != 'nan')]

In [ ]:
df[df['VAX_LOT3'] == 'EN6201']

In [ ]:
import matplotlib.pyplot as plt
# EN6201, FE6208
df[df['VAX_LOT2'] == 'EN6201'].hist(by=['VAX_LOT2'], column='SYMPTOMS', figsize=(200, 60))
# plt.savefig('EN6201.png')

In [ ]:
df['SYMPTOMS'].hist(by=df['VAX_LOT1'])

In [ ]:
df.hist(by=['VAX_LOT1'], column='SYMPTOMS')

In [ ]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt

grouped = df.groupby(['VAX_LOT1'])

for index, group in grouped:
  display(index, group)
  #plt.figure(figsize=(20, 10), edgecolor='green')
  #plt.title(index)
  #plt.hist(group['SYMPTOMS'], align='left')
  #plt.show()

In [ ]:
type(pd.NA)

In [ ]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt

x = ['A']*300 + ['B']*400 + ['C']*300
y = np.random.randn(1000)
df = DataFrame({'Letter':x, 'N':y})
grouped = df.groupby('Letter')

for index, group in grouped:
  display(group)
  plt.figure()
  plt.title(index)
  plt.hist(group.N)

# plt.show()

In [ ]:
symptomByBatchcodeTable.iloc[:1000].to_excel('tmp/symptomByBatchcodeTable.xlsx')

In [ ]:
internationalVaersCovid19 = getInternationalVaersCovid19(years = years_from_2020_to_present)
internationalVaersCovid19

In [ ]:
updateBatchCodeTableHtmlFile(internationalVaersCovid19, batchCodeTableHtmlFile="../docs/batchCodeTable.html")

In [ ]:
createAndSaveBatchCodeTables(internationalVaersCovid19, minADRsForLethality=100)